In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import vdmlab as vdm

from loading_data import get_data

In [ ]:
from run import test_infos
infos = test_infos

In [ ]:
# pickle_filepath = 'E:/code/emi_shortcut/cache/pickled'
pickle_filepath = 'C:/Users/Emily/code/emi_shortcut/cache/pickled'

In [ ]:
import info.r063d3 as info

tuning_curve_filename = info.session_id + '_tuning-curve.pkl'
pickled_tuning_curve = os.path.join(pickle_filepath, tuning_curve_filename)
with open(pickled_tuning_curve, 'rb') as fileobj:
    tuning_curve = pickle.load(fileobj)

In [ ]:
experiment_time = 'pauseA'
decode_filename = info.session_id + '_decode-' + experiment_time + '.pkl'
pickled_decode = os.path.join(pickle_filepath, decode_filename)
with open(pickled_decode, 'rb') as fileobj:
    decode = pickle.load(fileobj)

In [ ]:
decode['zones']['u'].n_samples

In [ ]:
from analyze_decode import analyze
experiment_time = 'phase1'
pausea = analyze(info, tuning_curve, experiment_time=experiment_time, shuffle_id=False)

In [ ]:
pausea['zones']['u'].n_samples

In [ ]:
position = pausea['decoded']

In [ ]:
pausea['zones']['shortcut'].n_samples

In [ ]:
pausea['zones']['novel'].n_samples

In [ ]:
pausea['times']

In [ ]:
11 / 358152

In [ ]:
pausea['decoded'].n_samples

In [ ]:
u = pausea['zones']['u']
shortcut = pausea['zones']['shortcut']
novel = pausea['zones']['novel']
other = pausea['zones']['other']

In [ ]:
position.n_samples, shortcut.n_samples, novel.n_samples, other.n_samples

In [ ]:
plt.plot(u.x, u.y, 'b.', ms=2)
plt.plot(shortcut.x, shortcut.y, 'g.', ms=2)
plt.plot(novel.x, novel.y, 'k.', ms=2)
plt.plot(other.x, other.y, 'r.', ms=2)
plt.show()

In [ ]:
binsize = 3
xedges = np.arange(position.x.min(), position.x.max() + binsize, binsize)
yedges = np.arange(position.y.min(), position.y.max() + binsize, binsize)
xx, yy = np.meshgrid(xedges, yedges)

histogram, xs, ys = np.histogram2d(position.x, position.y, bins=xx.shape)

pp = plt.pcolormesh(xx, yy, histogram, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
shuffle_id = False
from analyze_decode import get_edges, point_in_zones
from utils_maze import find_zones

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)

speed = position.speed(t_smooth=0.5)
run_idx = np.squeeze(speed.data) >= 0.1
run_pos = position[run_idx]

track_starts = [info.task_times['phase1'].start,
                info.task_times['phase2'].start,
                info.task_times['phase3'].start]
track_stops = [info.task_times['phase1'].stop,
               info.task_times['phase2'].stop,
               info.task_times['phase3'].stop]

track_pos = run_pos.time_slices(track_starts, track_stops)

binsize = 3
xedges = np.arange(position.x.min(), position.x.max() + binsize, binsize)
yedges = np.arange(position.y.min(), position.y.max() + binsize, binsize)

track_starts = [info.task_times['phase3'].start]
track_stops = [info.task_times['phase3'].stop]

track_pos = run_pos.time_slices(track_starts, track_stops)

if shuffle_id:
    random.shuffle(tuning_curve)

In [ ]:
position = track_pos

binsize = 3
xedges = np.arange(position.x.min(), position.x.max() + binsize, binsize)
yedges = np.arange(position.y.min(), position.y.max() + binsize, binsize)
xx, yy = np.meshgrid(xedges, yedges)

histogram, xs, ys = np.histogram2d(position.x, position.y, bins=xx.shape)

pp = plt.pcolormesh(xx, yy, histogram, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
len(spikes)

In [ ]:
if experiment_time == 'tracks':
    decode_spikes = [spiketrain.time_slices(track_starts, track_stops) for spiketrain in spikes]
    epochs_interest = vdm.Epoch(np.hstack([np.array(track_starts), np.array(track_stops)]))

else:
    decode_spikes = [spiketrain.time_slice(info.task_times[experiment_time].start,
                                           info.task_times[experiment_time].stop) for spiketrain in spikes]
#     sliced_lfp = lfp.time_slice(info.task_times[experiment_time].start, info.task_times[experiment_time].stop)
#     z_thresh = 3.0
#     power_thresh = 5.0
#     merge_thresh = 0.02
#     min_length = 0.01
#     swrs = vdm.detect_swr_hilbert(sliced_lfp, fs=info.fs, thresh=(140.0, 250.0), z_thresh=z_thresh,
#                                   power_thresh=power_thresh, merge_thresh=merge_thresh, min_length=min_length)

#     epochs_interest = vdm.find_multi_in_epochs(decode_spikes, swrs, min_involved=3)
#     if epochs_interest.n_epochs == 0:
#         epochs_interest = vdm.find_multi_in_epochs(decode_spikes, swrs, min_involved=1)

In [ ]:
len(decode_spikes)

In [ ]:
xy_centers.shape, time_centers.shape, likelihood.shape, decoding_tc.shape

In [ ]:
counts_binsize = 0.025
time_edges = get_edges(position, counts_binsize, lastbin=True)
counts = vdm.get_counts(decode_spikes, time_edges, gaussian_std=0.005)

decoding_tc = []
for tuning in tuning_curve:
    decoding_tc.append(np.ravel(tuning))
decoding_tc = np.array(decoding_tc)

likelihood = vdm.bayesian_prob(counts, decoding_tc, counts_binsize)

xcenters = (xedges[1:] + xedges[:-1]) / 2.
ycenters = (yedges[1:] + yedges[:-1]) / 2.
xy_centers = vdm.cartesian(xcenters, ycenters)

time_centers = (time_edges[1:] + time_edges[:-1]) / 2.

decoded = vdm.decode_location(likelihood, xy_centers, time_centers)
nan_idx = np.logical_and(np.isnan(decoded.x), np.isnan(decoded.y))
decoded = decoded[~nan_idx]

# if not decoded.isempty:
#     sequences = vdm.remove_teleports(decoded, speed_thresh=40, min_length=3)
# #     decoded_epochs = sequences.intersect(epochs_interest, boundaries=False)
#     decoded_epochs = sequences.intersect(vdm.Epoch(info.task_times[experiment_time].start, 
#                                                    info.task_times[experiment_time].stop), boundaries=False)
#     decoded = vdm.epoch_position(decoded, decoded_epochs)
# else:
#     raise ValueError("decoded cannot be empty.")

In [ ]:
decoded

In [ ]:
zones = find_zones(info, expand_by=7)
decoded_zones = point_in_zones(decoded, zones)

keys = ['u', 'shortcut', 'novel']
errors = dict()
actual_position = dict()
if experiment_time in ['phase1', 'phase2', 'phase3', 'tracks']:
    for trajectory in keys:
        actual_x = np.interp(decoded_zones[trajectory].time, track_pos.time, track_pos.x)
        actual_y = np.interp(decoded_zones[trajectory].time, track_pos.time, track_pos.y)
        actual_position[trajectory] = vdm.Position(np.hstack((actual_x[..., np.newaxis],
                                                              actual_y[..., np.newaxis])),
                                                   decoded_zones[trajectory].time)
        errors[trajectory] = actual_position[trajectory].distance(decoded_zones[trajectory])
else:
    for trajectory in decoded_zones:
        errors[trajectory] = []
        actual_position[trajectory] = []

output = dict()
output['zones'] = decoded_zones
output['errors'] = errors
output['times'] = len(time_centers)
output['actual'] = actual_position
output['decoded'] = decoded

In [ ]:
np.mean(output['errors']['u'])

In [ ]:
position = decoded

binsize = 3
xedges = np.arange(position.x.min(), position.x.max() + binsize, binsize)
yedges = np.arange(position.y.min(), position.y.max() + binsize, binsize)
xx, yy = np.meshgrid(xedges, yedges)

histogram, xs, ys = np.histogram2d(position.x, position.y, bins=xx.shape)

pp = plt.pcolormesh(xx, yy, histogram, cmap='pink_r')
plt.colorbar(pp)
plt.axis('off')
plt.show()

In [ ]:
decoded.n_samples, output['decoded'].n_samples

In [ ]:
len(time_centers)

In [ ]:
shuffle_id = False
experiment_time='pauseA'
from analyze_decode import get_edges, point_in_zones
from utils_maze import find_zones

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)

speed = position.speed(t_smooth=0.5)
run_idx = np.squeeze(speed.data) >= 0.1
run_pos = position[run_idx]

# track_starts = [info.task_times['phase1'].start,
#                 info.task_times['phase2'].start,
#                 info.task_times['phase3'].start]
# track_stops = [info.task_times['phase1'].stop,
#                info.task_times['phase2'].stop,
#                info.task_times['phase3'].stop]

# track_pos = run_pos.time_slices(track_starts, track_stops)

binsize = 3
xedges = np.arange(position.x.min(), position.x.max() + binsize, binsize)
yedges = np.arange(position.y.min(), position.y.max() + binsize, binsize)

track_starts = [info.task_times['phase3'].start]
track_stops = [info.task_times['phase3'].stop]

track_pos = run_pos.time_slices(track_starts, track_stops)

if shuffle_id:
    random.shuffle(tuning_curve)

if experiment_time == 'tracks':
    decode_spikes = [spiketrain.time_slices(track_starts, track_stops) for spiketrain in spikes]
    epochs_interest = vdm.Epoch(np.hstack([np.array(track_starts), np.array(track_stops)]))

else:
    decode_spikes = [spiketrain.time_slice(info.task_times[experiment_time].start,
                                           info.task_times[experiment_time].stop) for spiketrain in spikes]
    sliced_lfp = lfp.time_slice(info.task_times[experiment_time].start, info.task_times[experiment_time].stop)
    z_thresh = 3.0
    power_thresh = 5.0
    merge_thresh = 0.02
    min_length = 0.01
    swrs = vdm.detect_swr_hilbert(sliced_lfp, fs=info.fs, thresh=(140.0, 250.0), z_thresh=z_thresh,
                                  power_thresh=power_thresh, merge_thresh=merge_thresh, min_length=min_length)

    epochs_interest = vdm.find_multi_in_epochs(decode_spikes, swrs, min_involved=3)
    if epochs_interest.n_epochs == 0:
        epochs_interest = vdm.find_multi_in_epochs(decode_spikes, swrs, min_involved=1)

counts_binsize = 0.025
time_edges = get_edges(run_pos, counts_binsize, lastbin=True)
counts = vdm.get_counts(decode_spikes, time_edges, gaussian_std=0.005)

decoding_tc = []
for tuning in tuning_curve:
    decoding_tc.append(np.ravel(tuning))
decoding_tc = np.array(decoding_tc)

likelihood = vdm.bayesian_prob(counts, decoding_tc, counts_binsize)

xcenters = (xedges[1:] + xedges[:-1]) / 2.
ycenters = (yedges[1:] + yedges[:-1]) / 2.
xy_centers = vdm.cartesian(xcenters, ycenters)

time_centers = (time_edges[1:] + time_edges[:-1]) / 2.

decoded = vdm.decode_location(likelihood, xy_centers, time_centers)
nan_idx = np.logical_and(np.isnan(decoded.x), np.isnan(decoded.y))
decoded = decoded[~nan_idx]

if not decoded.isempty:
    sequences = vdm.remove_teleports(decoded, speed_thresh=40, min_length=3)
    decoded_epochs = sequences.intersect(epochs_interest, boundaries=False)
    decoded = vdm.epoch_position(decoded, decoded_epochs)
else:
    raise ValueError("decoded cannot be empty.")

zones = find_zones(info, expand_by=7)
decoded_zones = point_in_zones(decoded, zones)

keys = ['u', 'shortcut', 'novel']
errors = dict()
actual_position = dict()
if experiment_time == 'tracks':
    for trajectory in keys:
        actual_x = np.interp(decoded_zones[trajectory].time, track_pos.time, track_pos.x)
        actual_y = np.interp(decoded_zones[trajectory].time, track_pos.time, track_pos.y)
        actual_position[trajectory] = vdm.Position(np.hstack((actual_x[..., np.newaxis],
                                                              actual_y[..., np.newaxis])),
                                                   decoded_zones[trajectory].time)
        errors[trajectory] = actual_position[trajectory].distance(decoded_zones[trajectory])
else:
    for trajectory in decoded_zones:
        errors[trajectory] = []
        actual_position[trajectory] = []

output = dict()
output['zones'] = decoded_zones
output['errors'] = errors
output['times'] = len(time_centers)
output['actual'] = actual_position
output['decoded'] = decoded

In [ ]:
output['times']

In [ ]:
output['zones']['shortcut'].n_samples

In [ ]:
from utils_plotting import plot_intersects, plot_zone
plt.plot(position.x, position.y, 'k.', ms=1)
for zone in zones:
    if zones[zone].geom_type == 'MultiPolygon':
        plot_intersects(zones[zone])
    elif zones[zone].geom_type == 'Polygon':
        plot_zone(zones[zone])
    else:
        continue

In [ ]:
events, position, spikes, lfp, lfp_theta = get_data(info)

speed = position.speed(t_smooth=0.5)
run_idx = np.squeeze(speed.data) >= 0.1
run_pos = position[run_idx]

track_starts = [info.task_times['phase1'].start,
                info.task_times['phase2'].start,
                info.task_times['phase3'].start]
track_stops = [info.task_times['phase1'].stop,
               info.task_times['phase2'].stop,
               info.task_times['phase3'].stop]

track_pos = run_pos.time_slices(track_starts, track_stops)



In [ ]:
run_pos = position.time_slice(info.task_times['pauseA'].start, info.task_times['pauseA'].stop)

In [ ]:
binsize = 3
xedges = np.arange(position.x.min(), position.x.max() + binsize, binsize)
yedges = np.arange(position.y.min(), position.y.max() + binsize, binsize)

track_starts = [info.task_times['phase3'].start]
track_stops = [info.task_times['phase3'].stop]

In [ ]:
track_starts, track_stops

In [ ]:
shuffle_id = False
experiment_time = 'pauseA'



track_pos = run_pos.time_slices(track_starts, track_stops)

if shuffle_id:
    random.shuffle(tuning_curve)

if experiment_time == 'tracks':
    decode_spikes = [spiketrain.time_slices(track_starts, track_stops) for spiketrain in spikes]
    epochs_interest = vdm.Epoch(np.hstack([np.array(track_starts), np.array(track_stops)]))

else:
    decode_spikes = [spiketrain.time_slice(info.task_times[experiment_time].start,
                                           info.task_times[experiment_time].stop) for spiketrain in spikes]
    sliced_lfp = lfp.time_slice(info.task_times[experiment_time].start, info.task_times[experiment_time].stop)
    z_thresh = 3.0
    power_thresh = 5.0
    merge_thresh = 0.02
    min_length = 0.01
    swrs = vdm.detect_swr_hilbert(sliced_lfp, fs=info.fs, thresh=(140.0, 250.0), z_thresh=z_thresh,
                                  power_thresh=power_thresh, merge_thresh=merge_thresh, min_length=min_length)

    epochs_interest = vdm.find_multi_in_epochs(decode_spikes, swrs, min_involved=3)
    print(epochs_interest.n_epochs)
    if epochs_interest.n_epochs == 0:
        epochs_interest = vdm.find_multi_in_epochs(decode_spikes, swrs, min_involved=1)

In [ ]:
swrs.n_epochs

In [ ]:
epochs_interest.n_epochs

In [ ]:
from analyze_decode import get_edges

counts_binsize = 0.025
time_edges = get_edges(run_pos, counts_binsize, lastbin=True)
counts = vdm.get_counts(decode_spikes, time_edges, gaussian_std=0.005)

decoding_tc = []
for tuning in tuning_curve:
    decoding_tc.append(np.ravel(tuning))
decoding_tc = np.array(decoding_tc)

likelihood = vdm.bayesian_prob(counts, decoding_tc, counts_binsize)

In [ ]:
np.sum(np.isnan(likelihood)), likelihood.shape

In [ ]:
counts.shape

In [ ]:
run_pos.time[0], run_pos.time[-1]

In [ ]:
xcenters = (xedges[1:] + xedges[:-1]) / 2.
ycenters = (yedges[1:] + yedges[:-1]) / 2.
xy_centers = vdm.cartesian(xcenters, ycenters)

time_centers = (time_edges[1:] + time_edges[:-1]) / 2.

decoded = vdm.decode_location(likelihood, xy_centers, time_centers)
nan_idx = np.logical_and(np.isnan(decoded.x), np.isnan(decoded.y))
decoded = decoded[~nan_idx]

if not decoded.isempty:
    sequences = vdm.remove_teleports(decoded, speed_thresh=40, min_length=3)
    decoded_epochs = sequences.intersect(epochs_interest, boundaries=False)
    decoded = vdm.epoch_position(decoded, decoded_epochs)
else:
    raise ValueError("decoded cannot be empty.")

In [ ]:
decoded.n_samples

In [ ]:
from utils_maze import find_zones
from analyze_decode import point_in_zones

zones = find_zones(info, expand_by=7)
decoded_zones = point_in_zones(decoded, zones)

keys = ['u', 'shortcut', 'novel']
errors = dict()
actual_position = dict()
if experiment_time == 'tracks':
    for trajectory in keys:
        actual_x = np.interp(decoded_zones[trajectory].time, track_pos.time, track_pos.x)
        actual_y = np.interp(decoded_zones[trajectory].time, track_pos.time, track_pos.y)
        actual_position[trajectory] = vdm.Position(np.hstack((actual_x[..., np.newaxis],
                                                              actual_y[..., np.newaxis])),
                                                   decoded_zones[trajectory].time)
        errors[trajectory] = actual_position[trajectory].distance(decoded_zones[trajectory])
else:
    for trajectory in decoded_zones:
        errors[trajectory] = []
        actual_position[trajectory] = []

output = dict()
output['zones'] = decoded_zones
output['errors'] = errors
output['times'] = len(time_centers)
output['actual'] = actual_position
output['decoded'] = decoded

In [ ]:
output['times']

In [ ]:
51 / 358152

In [ ]:
output['decoded'].n_samples

In [ ]:
output['zones']['u'].n_samples

In [ ]:
dec = output['decoded']

In [ ]:
np.mean(output['errors']['u']), np.mean(output['errors']['shortcut']), np.mean(output['errors']['novel'])

In [ ]:
plt.plot(dec.x, dec.y, 'b.', ms=2)
plt.show()

In [ ]:
output['zones']['u'].n_samples + output['zones']['shortcut'].n_samples + output['zones']['novel'].n_samples

In [ ]:
dec.n_samples

In [ ]:
# saving many tuning curves on the same plot
import matplotlib.gridspec as gridspec

rows = 8
cols = 14
fig = plt.figure(figsize=(10, 10))

grids = gridspec.GridSpec(cols, rows)
grids.update(wspace=0.02, hspace=0.05)

xx, yy = np.meshgrid(xedges, yedges)
for i in range(rows*cols):
    ax = plt.subplot(grids[i])
    pp = ax.pcolormesh(xx, yy, tuning_curves[i], cmap='pink_r')
    ax.axis('off')
    
plt.show()

# savepath = 'E:/code/emi_shortcut/plots/tuning/' + info.session_id + 'many_tuning-curves.png'
# plt.savefig(savepath, dpi=300, bbox_inches='tight')

# Testing Neuralynx events

In [ ]:
dataloc = 'C:/Users/EmilyWork/Desktop/20170104'
session = '2017-01-05_test1'
event_filename = 'Events.nev'
# event_labels = dict(on='on',
#                     off='off')
event_labels = dict(on='TTL Output on AcqSystem1_0 board 0 port 3 value (0x0002).',
                    off='TTL Output on AcqSystem1_0 board 0 port 3 value (0x0000).')

In [ ]:
filepath = os.path.join(dataloc, session, event_filename)
events = vdm.load_events(filepath, event_labels)

In [ ]:
len(events['off']), len(events['on'])